In [1]:
# This makes a first contrast curve for Altair

# created 2018 Sept. 20 by E.S.

## SECTION TO INITIALIZE

In [1]:
# import stuff

import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ContrastCurveModule, \
                                       FluxAndPosition, \
                                    StackingAndSubsampling
from PynPoint.ProcessingModules.StackingAndSubsampling import DerotateAndStackModule
from PynPoint.ProcessingModules.FluxAndPosition import FakePlanetModule
#from PynPoint.Util import AnalysisTools

In [2]:
# define workspaces and initialize Pypeline

stem = "/home/../../media/unasemaje/Elements/lbti_data_reduction/180507_fizeau_altair/pynpoint_testing/"

working_place = stem+"pynpoint_experimentation_altair/working_place/"
input_place = stem+"/pynpoint_experimentation_altair/input_place/"
output_place = stem+"/pynpoint_experimentation_altair/output_place/"

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


In [3]:
## IF THE *INI FILE WAS NEWLY GENERATED, EDIT THE *INI FILE TO SET
## PIXSCALE = 0.0107

## SECTION TO READ IN DATA

In [4]:
# read in science FITS files 
# (should have PARANG=0 in all headers, so as to keep the PSF in the same original orientation on the array; 
# we need to wait to correct for PARANG later)

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [5]:
# confirm PIXSCALE change

pixscale_config = pipeline.get_attribute("config", "PIXSCALE")
print("Plate scale for /config/ is "+str(pixscale_config))
#pixscale_sci = pipeline.get_attribute("science", "PIXSCALE")
#print("Plate scale for /science/ is "+str(pixscale_sci))

Plate scale for /config/ is 0.0107


In [6]:
# read in PSF reference FITS files (i.e., unsaturated frames)
# (these do not have any PARANG in the header, but it probably doesn't matter)

read_ref_psf = FitsReadingModule(name_in="read_ref_psf",
                                 input_dir=input_place+'ref_psf/',
                                 image_tag="ref_psf",
                                 check=True)

pipeline.add_module(read_ref_psf)

## SECTION TO PROCESS DATA

In [8]:
## PIPELINE CHANNEL 1: IMAGE RESIDUALS

# generate PCA basis from unsaturated frames and
# do PCA PSF subtraction of the saturated frames

'''
pca_pca_subt = PcaPsfSubtractionModule(pca_numbers=(5, ),
                                       name_in="pca",
                                       images_in_tag="science",
                                       reference_in_tag="ref_psf",
                                       res_mean_tag="mean_residuals",
                                       res_median_tag="median_residuals",
                                       res_arr_out_tag="all_resids",
                                       res_rot_mean_clip_tag="resid_rot",
                                       basis_out_tag="pca_components",
                                       subtract_mean=True,
                                       verbose=True)

pipeline.add_module(pca_pca_subt)
'''

# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images

In [7]:
## PIPELINE CHANNEL 2: MAKE CONTRAST CURVE

# make a contrast curve 
# (N.b. this does not separately require PcaPsfSubtractionModule)

'''
cent_size: mask radius
'''

# N.b. scale the reference PSF by 3.28 to match Altair's amplitude

contrast_curve = ContrastCurveModule(name_in="contrast_curve",
                            image_in_tag="science",
                            psf_in_tag="ref_psf",
                            contrast_out_tag="contrast_landscape",
                            pca_out_tag="pca_resids",
                            pca_number=20,
                            psf_scaling=3.28,
                            separation=(0.05, 0.45, 0.05), 
                            angle=(0.0, 360.0, 60.0), 
                            magnitude=(7.5, 1.0),
                            cent_size=None)

pipeline.add_module(contrast_curve)

In [8]:
# write out PCA residuals

write_pca_resids = FitsWritingModule(file_name="second_pass_pca_resids.fits",
                              name_in="write_pca_resids",
                              output_dir=output_place,
                              data_tag="pca_resids")

pipeline.add_module(write_pca_resids)

In [9]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute INSTRUMENT (=INSTRUME) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute DATE (=DATE-OBS) not found in the FITS header.
  "header." % (item[0], fitskey))


Running FitsReadingModule... [DONE]
Running ContrastCurveModule...
Processing position 1 out of 48.

...

..
Processing position 15 out of 48.

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:350: UserWarning: ContrastModule could not converge at the position of 0.19999999999999998 arcsec and 60.0 deg.
  "%s arcsec and %s deg." % (sep*pixscale, ang))


..............................
Processing position 16 out of 48...........................

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:327: UserWarning: Magnitude decreases but false positive fraction increases. Adjusting magnitude to 2.123448621861486 and step size to 0.05
  "to %s" % (list_mag[-3], mag_step/2.))



Processing position 17 out of 48...........
Processing position 18 out of 48.........
Processing position 19 out of 48...........
Processing position 20 out of 48..............
Processing position 21 out of 48.....................
Processing position 22 out of 48......
Processing position 23 out of 48.....
Processing position 24 out of 48....
Processing position 25 out of 48........
Processing position 26 out of 48....................
Processing position 27 out of 48.....
Processing position 28 out of 48.......
Processing position 29 out of 48.........
Processing position 30 out of 48..
Processing position 31 out of 48.......
Processing position 32 out of 48....................
Processing position 33 out of 48...
Processing position 34 out of 48.........
Processing position 35 out of 48........
Processing position 36 out of 48.........
Processing position 37 out of 48........
Processing position 38 out of 48..........
Processing position 39 out of 48.......
Processing position 40 out 

In [ ]:
#####################################################################################################################

In [ ]:
# extract contrast curve data

In [10]:
contrast_curve_results = pipeline.get_data("contrast_landscape")
print(contrast_curve_results)

[[1.00000000e-01 1.09212109e+00 1.02641282e-01 3.74521694e-03]
 [1.50000000e-01 2.27194406e+00 7.25722515e-03 7.82638977e-04]
 [2.00000000e-01 3.36152977e+00 1.72031808e+00 2.68666801e-04]
 [2.50000000e-01 4.18260090e+00 2.93652847e-01 1.21477003e-04]
 [3.00000000e-01 5.45496897e+00 1.92776441e-01 6.54338490e-05]
 [3.50000000e-01 5.95574646e+00 1.45560487e-01 3.97498750e-05]
 [4.00000000e-01 6.09150519e+00 3.10658553e-02 2.33216658e-05]
 [4.50000000e-01 6.38083668e+00 3.10725530e-02 1.68024150e-05]]


In [15]:
# make plots

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(contrast_curve_results[:,0],contrast_curve_results[:,1])
ax1.set_title('Azim. averaged contrast limit')
ax1.set_xlabel('Radius (asec)')
ax1.set_ylabel('Radius (asec)')
ax1.invert_yaxis()
ax2.set_ylabel('del_mag')
ax2.plot(contrast_curve_results[:,0],contrast_curve_results[:,3])
ax2.set_title('Threshold of FPF')
ax2.set_xlabel('Radius (asec)')
plt.tight_layout()
f.savefig('test.png')